In [133]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import cv2 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from skimage.filters import threshold_otsu
import seaborn as sns
from skimage.color import rgb2gray
from skimage.morphology import convex_hull_image
from scipy.spatial import ConvexHull
from skimage.measure import label, regionprops
from skimage.draw import polygon,rectangle,polygon_perimeter
from scipy.stats import f_oneway
from itertools import combinations
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
from scipy.stats import ttest_1samp

In [134]:
dataset1 = pd.read_csv(r'.\data_2\dataset.csv')  
dataset2 = pd.read_csv(r'.\data_3\Ensemble.csv')  

In [135]:
dataset1.drop('Feature9',axis=1,inplace=True)

In [136]:
# Generate the years based on the number of rows and rows per year
start_year = 1984
rows_per_year = 8
total_rows = len(dataset1)

# Calculate the year for each row
years = [start_year + (i // rows_per_year) for i in range(total_rows)]

# Add the 'Year' column to the dataset
dataset1['Year'] = years

# Print the first few rows to verify

dataset1.tail()

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Year
295,0.1387,92597.5043,90066.2099,1181.6302,449,1093,0.7577,0.1645,2020
296,0.1386,92571.9423,90039.6155,1181.6302,449,1093,0.7579,0.1644,2021
297,0.1386,92571.9423,90039.6155,1181.6302,449,1093,0.7579,0.1644,2021
298,0.1387,92597.5043,90066.2099,1181.6302,449,1093,0.7577,0.1645,2021
299,0.1386,92571.9423,90039.6155,1181.6302,449,1093,0.7579,0.1644,2021


In [137]:
dataset2.tail()

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Year
35,0.123793,87567.126940,85472.840950,1173.521445,430.812764,1091.894540,0.773433,0.144715,2026
36,0.122943,87437.723171,85034.457589,1171.168381,428.380861,1090.489054,0.772514,0.144659,2026
37,0.124064,87690.057250,85364.862096,1169.257333,428.803109,1088.566045,0.771956,0.146680,2026
38,0.123310,87586.776885,85180.549198,1168.388857,426.472141,1088.139617,0.771116,0.143883,2026
39,0.123556,87473.157734,85042.395840,1167.182011,426.815116,1086.638589,0.770828,0.144143,2026


In [138]:
dataset1.shape

(300, 9)

In [139]:
data = pd.concat([dataset1, dataset2], axis=0, ignore_index=True)

In [140]:
# Add 4 empty columns
data['mass1'] = 0
data['mass2'] = 0
data['sea1'] = 0
data['sea2'] = 0

In [141]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Feature1  340 non-null    float64
 1   Feature2  340 non-null    float64
 2   Feature3  340 non-null    float64
 3   Feature4  340 non-null    float64
 4   Feature5  340 non-null    float64
 5   Feature6  340 non-null    float64
 6   Feature7  340 non-null    float64
 7   Feature8  340 non-null    float64
 8   Year      340 non-null    int64  
 9   mass1     340 non-null    int64  
 10  mass2     340 non-null    int64  
 11  sea1      340 non-null    int64  
 12  sea2      340 non-null    int64  
dtypes: float64(8), int64(5)
memory usage: 34.7 KB


In [142]:
# Filter the dataset for the years 2019 to 2023
mask = (data['Year'] >= 2019) & (data['Year'] <= 2023)

# Assign the specified values for mass1, mass2, sea1, and sea2 in this range
data.loc[mask, 'mass1'] = -57.11
data.loc[mask, 'mass2'] = 7.68
data.loc[mask, 'sea1'] = -0.158
data.loc[mask, 'sea2'] = 0.021

C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3134289412.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-57.11' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[mask, 'mass1'] = -57.11
C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3134289412.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7.68' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[mask, 'mass2'] = 7.68
C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3134289412.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.158' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[mask, 'sea1'] = -0.158
C:\Users\Roumo\AppData\Loca

In [143]:
# Filter the dataset for the years 2019 to 2023
mask = (data['Year'] >= 1986) & (data['Year'] <= 2016)

# Assign the specified values for mass1, mass2, sea1, and sea2 in this range
data.loc[mask, 'mass1'] = -3.84
data.loc[mask, 'mass2'] = 0.50

In [144]:
# Load the dataset
df=data
# Filter out rows where mass1, mass2, sea1, or sea2 contain 0
known_data = df[(df['mass1'] != 0) | (df['mass2'] != 0) | (df['sea1'] != 0) | (df['sea2'] != 0)]

In [145]:
known_data

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Year,mass1,mass2,sea1,sea2
16,0.189800,108056.171200,100616.941300,1250.386300,479.000000,1155.000000,0.729400,0.239000,1986,-3.84,0.50,0.000,0.000
17,0.190000,108100.045000,100534.706300,1250.386300,479.000000,1155.000000,0.729800,0.239300,1986,-3.84,0.50,0.000,0.000
18,0.189900,108062.316100,100513.808600,1250.386300,479.000000,1155.000000,0.729900,0.239200,1986,-3.84,0.50,0.000,0.000
19,0.189900,108060.437400,100510.254300,1250.386300,479.000000,1155.000000,0.729900,0.239100,1986,-3.84,0.50,0.000,0.000
20,0.189900,108072.412600,100511.752200,1250.386300,479.000000,1155.000000,0.729900,0.239200,1986,-3.84,0.50,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,0.120528,86435.318364,83060.157926,1165.755355,422.441557,1087.223625,0.778194,0.140505,2023,-57.11,7.68,-0.158,0.021
312,0.120388,86645.523970,83330.382443,1165.454713,422.577130,1086.615780,0.777967,0.141377,2023,-57.11,7.68,-0.158,0.021
313,0.121079,86397.169083,83320.472785,1164.347776,422.266998,1086.805607,0.778408,0.139873,2023,-57.11,7.68,-0.158,0.021
314,0.120143,86553.439427,83317.579144,1163.743132,421.293337,1085.450700,0.777941,0.140357,2023,-57.11,7.68,-0.158,0.021


In [146]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge


l:list = ['Feature1', 'Feature2', 'Feature3', 'Feature4', 'Feature5', 'Feature6',
        'Feature8']

# Split the data into features and targets
features = known_data[l]  # 'Year' column as feature


# lr_mass1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=3)
# lr_mass2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=4)
# lr_sea1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=4)
# lr_sea2 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.01, max_depth=4) 


# Initialize the LinearRegression model
# lr_mass1 = Ridge(alpha=1.0)
# lr_mass2 = Ridge(alpha=1.0)
# lr_sea1 = Ridge(alpha=1.0)
# lr_sea2 = Ridge(alpha=1.0)


# Initialize the LinearRegression model
lr_mass1 = LinearRegression()
lr_mass2 = LinearRegression()
lr_sea1 = LinearRegression()
lr_sea2 = LinearRegression()

# Separate target columns
target_mass1 = known_data['mass1']
target_mass2 = known_data['mass2']
target_sea1 = known_data['sea1']
target_sea2 = known_data['sea2']


# Train the models
lr_mass1.fit(features, target_mass1)
lr_mass2.fit(features, target_mass2)
lr_sea1.fit(features, target_sea1)
lr_sea2.fit(features, target_sea2)

# Now, prepare data for prediction (2024-2026)
predict_data = df[df['Year'] >= 2024]  # Filter the years to predict

# Prepare the 'Year' column for prediction
predict_years = predict_data[l]

# Predict the missing values for mass1, mass2, sea1, sea2
predicted_mass1 = lr_mass1.predict(predict_years)
predicted_mass2 = lr_mass2.predict(predict_years)
predicted_sea1 = lr_sea1.predict(predict_years)
predicted_sea2 = lr_sea2.predict(predict_years)

# Assign the predicted values to the prediction data
predict_data['mass1'] = predicted_mass1
predict_data['mass2'] = predicted_mass2
predict_data['sea1'] = predicted_sea1
predict_data['sea2'] = predicted_sea2


C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3412760911.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['mass1'] = predicted_mass1
C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3412760911.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['mass2'] = predicted_mass2
C:\Users\Roumo\AppData\Local\Temp\ipykernel_10660\3412760911.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [147]:
predict_data

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Year,mass1,mass2,sea1,sea2
316,0.119236,85937.068370,82633.683797,1161.661486,417.711780,1083.955151,0.778505,0.138190,2024,-7.585151,1.004790,-0.011108,0.001476
317,0.118742,85509.831245,82543.834165,1161.584175,417.295287,1082.318264,0.778572,0.137690,2024,9.559451,-1.306046,0.039743,-0.005282
318,0.118099,85564.677956,82599.906761,1160.180105,417.423642,1081.207179,0.779339,0.137462,2024,-7.794960,1.033069,-0.011730,0.001559
319,0.117339,85304.745549,82312.295511,1155.325745,416.479469,1076.754455,0.779033,0.136387,2024,-9.509974,1.264228,-0.016817,0.002235
320,0.116320,84971.773404,82076.552318,1150.825957,414.439897,1076.169839,0.778336,0.135506,2024,-29.196535,3.917682,-0.075208,0.009996
321,0.116334,84749.371631,82028.898359,1153.215066,417.444155,1075.587143,0.778911,0.134262,2024,-3.317096,0.429520,0.001551,-0.000206
322,0.116278,85026.143237,82361.422861,1156.325905,419.578893,1077.578225,0.780978,0.134806,2024,-17.049768,2.280479,-0.039180,0.005208
323,0.116028,84921.065715,82054.379551,1156.817114,422.708755,1077.594766,0.782989,0.134844,2024,-21.921599,2.937129,-0.053630,0.007128
324,0.115798,84676.101481,81909.957370,1156.027823,423.252469,1076.142576,0.783378,0.134510,2025,-13.460277,1.796670,-0.028534,0.003792
325,0.115287,84853.464587,81942.526850,1156.449836,423.574589,1076.353088,0.784833,0.134494,2025,-31.064681,4.169480,-0.080749,0.010732


In [152]:
predict_data.describe().T.drop(["25%"	,"50%"	,"75%",'count'],axis=1)

,mean,std,min,max
Feature1,0.118814,0.003048,0.115287,0.124064
Feature2,85877.361392,1091.313538,84676.101481,87690.057250
Feature3,83201.211444,1280.378299,81909.957370,85472.840950
Feature4,1163.564403,7.180705,1150.825957,1175.327077
Feature5,423.961854,5.050922,414.439897,430.812764
Feature6,1083.632384,6.072926,1075.587143,1093.423519
Feature7,0.779381,0.004599,0.770828,0.784915
Feature8,0.138559,0.004116,0.134262,0.146680
Year,2025.000000,0.834058,2024.000000,2026.000000
mass1,-19.628064,12.476444,-45.370417,9.559451


In [124]:
predict_data = predict_data.groupby('Year').mean()

In [125]:
predict_data[['mass1','mass2','sea1','sea2']]

,mass1,mass2,sea1,sea2
Year,,,,
2024,-10.851954,1.445107,-0.020798,0.002764
2025,-19.321437,2.586666,-0.045918,0.006103
2026,-28.710802,3.852212,-0.073767,0.009805
